In [599]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse

In [600]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [601]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [623]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 23 columns):
 #   Column                                              Non-Null Count   Dtype   
---  ------                                              --------------   -----   
 0   Episode_Title                                       250000 non-null  category
 1   Episode_Length_minutes                              250000 non-null  float64 
 2   Genre                                               250000 non-null  category
 3   Host_Popularity_percentage                          250000 non-null  float64 
 4   Publication_Day                                     250000 non-null  category
 5   Publication_Time                                    250000 non-null  category
 6   Guest_Popularity_percentage                         250000 non-null  float64 
 7   Number_of_Ads                                       250000 non-null  float64 
 8   Episode_Sentiment                                   25

In [603]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [604]:
train.drop('Podcast_Name', inplace=True, axis=1)
test.drop('Podcast_Name', inplace=True, axis=1)

In [605]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [606]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [607]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [608]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [609]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [610]:
x = x.iloc[:,:23]
test = test.iloc[:,:23]

In [611]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 23 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Episode_Title                                       750000 non-null  float64
 1   Episode_Length_minutes                              750000 non-null  float64
 2   Genre                                               750000 non-null  float64
 3   Host_Popularity_percentage                          750000 non-null  float64
 4   Publication_Day                                     750000 non-null  float64
 5   Publication_Time                                    750000 non-null  float64
 6   Guest_Popularity_percentage                         750000 non-null  float64
 7   Number_of_Ads                                       750000 non-null  float64
 8   Episode_Sentiment                                   750000 non-n

In [612]:
mm_scaler = skp.MinMaxScaler()
x_s = pd.DataFrame(mm_scaler.fit_transform(x), columns=x.columns, index=x.index)

In [613]:
for c in categories: 
    x_s[c] = x_s[c].astype('category')
    test[c] = test[c].astype('category')

In [ ]:
K = 100
kf = KFold(K, shuffle=True, random_state=55)

In [639]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [640]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x_s.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x_s.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist")#, min_child_weight=4)
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)
    #xgb.plot_importance(model.get_booster().get_score(importance_type='weight'), show_values=True)

[0]	validation_0-rmse:21.21659
[1]	validation_0-rmse:17.61506
[2]	validation_0-rmse:15.53313
[3]	validation_0-rmse:14.39759
[4]	validation_0-rmse:13.79563
[5]	validation_0-rmse:13.48385
[6]	validation_0-rmse:13.32265
[7]	validation_0-rmse:13.23560
[8]	validation_0-rmse:13.19099
[9]	validation_0-rmse:13.16337
[10]	validation_0-rmse:13.14736
[11]	validation_0-rmse:13.13764
[12]	validation_0-rmse:13.12890
[13]	validation_0-rmse:13.11974
[14]	validation_0-rmse:13.11533
[15]	validation_0-rmse:13.11325
[16]	validation_0-rmse:13.11124
[17]	validation_0-rmse:13.10884
[18]	validation_0-rmse:13.10669
[19]	validation_0-rmse:13.10390
[20]	validation_0-rmse:13.10144
[21]	validation_0-rmse:13.10083
[22]	validation_0-rmse:13.10013
[23]	validation_0-rmse:13.09866
[24]	validation_0-rmse:13.09706
[25]	validation_0-rmse:13.09660
[26]	validation_0-rmse:13.09632
[27]	validation_0-rmse:13.09743
[28]	validation_0-rmse:13.09543
[29]	validation_0-rmse:13.09458
[30]	validation_0-rmse:13.09470
[31]	validation_0-

In [657]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [658]:
pred.head()

,0,0,0
750000,87.069580,92.941681,94.649338
750001,100.098572,108.450378,102.920135
750002,93.879654,84.125229,87.776176
750003,103.354164,119.403305,97.170471
750004,109.613914,110.228149,103.943336


In [664]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000     91.553528
750001    103.823029
750002     88.593689
750003    106.642647
750004    107.928467
Name: Listening_Time_minutes, dtype: float32

In [ ]:
pred_sum.to_csv('submission.csv', index_label='id')